In [5]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle
from multiprocessing import Pool
from runpy import run_path

import time
import sys

# ABPRED_DIR = Path.cwd().parent
# if ABPRED_DIR not in sys.path:
#     sys.path.append(str(ABPRED_DIR))
    
from AbPred.interface_contacts import InterfaceContacts
from AbPred.features_ic import ArpeggioFeatures,features15,features300,features400
from AbPred.FoldX import FoldX
from features3D.Structural import run_pops
from features3D.Structural import POPS_parse2
from AbPred.FoldX import names_stability_complex_wt,names_stability_complex_mut
from AbPred.FoldX import read_analyse_complex
from AbPred.FoldX import convert_features_to_differences
from biopandas.pdb import PandasPdb
import prody

In [6]:
# adapt benchmark dataframe
adapt_dataframe = pd.read_csv('../data/3bdy_adapt.csv',sep=";")
adapt_dataframe = adapt_dataframe.loc[-adapt_dataframe.Mutation.str.contains('S30b|S30d')]

In [7]:
# PDB files
PDBS_MUT = Path("../benchmark/3dby_adapt/")


pdbs_mut = list(PDBS_MUT.glob('*Repair4.pdb'))

# Clean structures to run arpeggio

In [8]:
CLEANPDB = "../../AbPred/libs/tools/clean_pdb.py"

In [9]:
def run_cleanpdb(pdb):
    sys.argv = ["",pdb]
    run_path(str(CLEANPDB),run_name="__main__")

In [10]:
with Pool(3) as p:
    print(p.map(run_cleanpdb, [str(pdb) for pdb in pdbs_mut]))

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  w

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE) with given element

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OH) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NZ) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE1) with given element '

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OE2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE2) with given element 

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'S' for Atom (name=SG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CZ2) with given element

...

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'S' for Atom (name=SG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND2) with given element

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NZ) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD2) with given element 

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NZ) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OG1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OH) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND1) with given element 

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'O' for Atom (name=OH) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=ND1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'N' for Atom (name=NE1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CE3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning: Used element 'C' for Atom (name=CZ2) with given element

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [11]:
pdbs_clean_mut = list(PDBS_MUT.glob('*clean.pdb'))

In [12]:
pdbs_clean_mut

[PosixPath('../benchmark/3dby_adapt/3bdy.protein.H31G.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H31K.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H31Q.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H31R.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H31S.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H33M.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H33R.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H33W.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H52R.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H52W.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H53K.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H53R.Repair4.clean.pdb'),
 PosixPath('../benchmark/3dby_adapt/3bdy.protein.H98A.Repair4.clean.pdb'),
 PosixPath('../benchmark/